In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [81]:
import os
import time
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler, random_split, WeightedRandomSampler
from torchvision.datasets import ImageFolder
from torchvision import models
import torch.nn as nn
import torch.optim as optim

In [84]:
# Constants
DATA_DIR = '/content/drive/MyDrive/JPEGImages'
BATCH_SIZE = 64
NUM_WORKERS = 8
PIN_MEMORY = True
TRAINING_EPOCHS = 10
LEARNING_RATE = 0.001

# Data loading and preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images
])

# Load the dataset
dataset = ImageFolder(root=DATA_DIR, transform=transform)


In [85]:
# Calculate class weights for balancing training data
class_counts = torch.tensor([label for label in dataset.targets]).bincount().float()
class_weights = 1.0 / (class_counts + 1e-6)  # Add a small constant to avoid division by zero
sample_weights = class_weights[torch.tensor(dataset.targets)]

# Create a sampler for weighted sampling
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Split dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_indices, test_indices = random_split(range(len(dataset)), [train_size, test_size])

# Create subset samplers for train and test to avoid index error
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Data loaders
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler,
                          num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler,
                         num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)

In [86]:
# Model definition
model = models.resnet50(pretrained=True)  # Use a pretrained ResNet-50 model
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers

# Add custom classifier
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 50),  # Assuming 50 animal classes
    nn.BatchNorm1d(50),
    nn.LogSoftmax(dim=1)
)

# Model training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()  # For mixed precision training

In [87]:
# Model training
for epoch in range(TRAINING_EPOCHS):
    model.train()
    start_time = time.time()  # Start timing the training
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if (i + 1) % 20 == 0:
            print(f'Epoch [{epoch + 1}/{TRAINING_EPOCHS}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    end_time = time.time()  # End timing the training
    print(f"Training time for epoch {epoch + 1}: {end_time - start_time:.2f} seconds")





Epoch [1/10], Step [20/129], Loss: 1.9514
Epoch [1/10], Step [40/129], Loss: 1.5154
Epoch [1/10], Step [60/129], Loss: 1.5276
Epoch [1/10], Step [80/129], Loss: 1.3371
Epoch [1/10], Step [100/129], Loss: 1.4626
Epoch [1/10], Step [120/129], Loss: 1.0899
Training time for epoch 1: 28.32 seconds
Epoch [2/10], Step [20/129], Loss: 1.1351
Epoch [2/10], Step [40/129], Loss: 0.9294
Epoch [2/10], Step [60/129], Loss: 1.0616
Epoch [2/10], Step [80/129], Loss: 0.9491
Epoch [2/10], Step [100/129], Loss: 1.1489
Epoch [2/10], Step [120/129], Loss: 0.7633
Training time for epoch 2: 27.78 seconds
Epoch [3/10], Step [20/129], Loss: 0.9612
Epoch [3/10], Step [40/129], Loss: 1.0638
Epoch [3/10], Step [60/129], Loss: 0.8275
Epoch [3/10], Step [80/129], Loss: 1.0207
Epoch [3/10], Step [100/129], Loss: 0.9564
Epoch [3/10], Step [120/129], Loss: 1.0157
Training time for epoch 3: 28.22 seconds
Epoch [4/10], Step [20/129], Loss: 0.7981
Epoch [4/10], Step [40/129], Loss: 1.0425
Epoch [4/10], Step [60/129], Lo

In [88]:
# Model evaluation
model.eval()
total = 0
correct = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')


Accuracy of the model on the test images: 90.58%
